# 2019320016 차주한

### Load dataset

In [1]:
import pandas as pd

data = pd.read_csv("./data/train.csv")
del data["id"]

print(len(data))
print(data[:5])

data.drop_duplicates(subset=["mail"], inplace=True)
print(len(data))

3620
   label                                               mail
0      0  Subject: update on prc process\r\nplease read ...
1      1  Subject: via - gra pro will get you hard consu...
2      0  Subject: re : southern\r\ndarren :\r\ni zeroed...
3      0  Subject: hpl nom for sept . 22 , 2000\r\n( see...
4      1  Subject: perform your best in bed\r\ngeneric c...
3528


### Split train and validation set

In [2]:
from sklearn.model_selection import train_test_split

X = data["mail"]
y = data["label"]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

print("train data")
print(f"non spam: {round(y_train.value_counts()[0]/len(y_train) * 100, 3)}%")
print(f"spam: {round(y_train.value_counts()[1]/len(y_train) * 100, 3)}%")

print()

print("validation data")
print(f"non spam: {round(y_val.value_counts()[0]/len(y_val) * 100, 3)}%")
print(f"non spam: {round(y_val.value_counts()[1]/len(y_val) * 100, 3)}%")

train data
non spam: 71.297%
spam: 28.703%

validation data
non spam: 71.246%
non spam: 28.754%


### Preprocess data

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

vectorize = CountVectorizer(stop_words="english")
X_train_dtm = vectorize.fit_transform(X_train)
print(X_train_dtm.shape)

(2822, 35143)


### Train naive baise model

In [7]:
from sklearn.naive_bayes import MultinomialNB

mod = MultinomialNB(alpha=1.0)
mod.fit(X_train_dtm, y_train)

MultinomialNB()

### Test on validation set

In [9]:
from sklearn.metrics import accuracy_score

X_val_dtm = vectorize.transform(X_val)
predicted = mod.predict(X_val_dtm)

print(accuracy_score(y_val, predicted))

0.9745042492917847


### Predict on test set

In [12]:
import csv

test_data = pd.read_csv("./data/test.csv")
del test_data["id"]

print(len(test_data))
print(test_data[:5])

X_test = test_data["mail"]
X_test_dtm = vectorize.transform(X_test)
predicted = mod.predict(X_test_dtm)

print(predicted)

f = open("result_NB.csv", "w", newline="")
wr = csv.writer(f)
wr.writerow(["id", "label"])

id = 0
for i in predicted:
    wr.writerow([id, i])
    id += 1

f.close()

1551
                                                mail
0  Subject: re : coastal ctr # 96008903 meter 098...
1  Subject: re :\r\nfyi\r\n- - - - - - - - - - - ...
2  Subject: hpl nomination changes for july 25 an...
3  Subject: new stack manager\r\nthere is a new v...
4  Subject: deletion of your enrononline user id\...
[0 0 0 ... 1 0 0]
